In [2]:
import os

In [5]:
os.getcwd()

'e:\\Mlop\\End to end\\Chicken_desease_Classification_'

In [4]:
os.chdir("../")

In [6]:
import tensorflow as tf

In [7]:
model =("artifacts\training\model.keras")

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path    
    training_data: Path      
    all_parameters: dict  
    params_image_size: list          
    params_batch_size:int 

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml , create_directories ,save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_pathfile= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):
        self.config=read_yaml(config_pathfile) 
        self.params=read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])
        
        
        
    def get_validation_config(self) ->EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model="artifacts/training/model.keras",    
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_parameters=self.params,  
            params_image_size=self.params.IMAGE_SIZE,          
            params_batch_size=self.params.BATCH_SIZE 
        )
        
        return eval_config

In [13]:
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        # Prepare the validation data generator with image rescaling and validation split
        datagenerator_kwargs = dict(rescale=1. / 255, validation_split=0.30)
        
        # Additional parameters for the dataflow
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Resize images to target size (excluding channels)
            batch_size=self.config.params_batch_size,  # Batch size from config
            interpolation="bilinear"  # Interpolation method for resizing images
        )

        # Initialize the ImageDataGenerator for validation
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        # Create a validation data generator by loading images from the directory
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),  # Directory for training data
            subset="validation",  # We are only using the validation subset
            shuffle=False,  # Don't shuffle to maintain consistency during evaluation
            **dataflow_kwargs  # Apply the dataflow arguments for resizing, batch size, etc.
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        # Load a pre-trained model from the given path
        return tf.keras.models.load_model(path)

    def evaluation(self):
        # Load the model to be evaluated
        self.model = self.load_model(self.config.path_of_model)  # Corrected to use `load_model` method
        self._valid_generator()  # Prepare the validation data generator
        self.score = self.model.evaluate(self.valid_generator)  # Evaluate the model on the validation data

    def save_score(self):
        # Save the evaluation scores (loss and accuracy) into a JSON file
        scores = {'loss': self.score[0], "accuracy": self.score[1]}  # Fix the reference to `score[1]` for accuracy
        save_json(path=Path("scores.json"), data=scores)  # Save the scores to a JSON file


In [15]:
try:
    config= ConfigurationManager()
    val_config =config.get_validation_config()
    evaluation =Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e    

[2025-02-27 01:30:03,810] common.py - 31 root - INFO - yaml file: config\config.yaml loaded successfully
[2025-02-27 01:30:03,825] common.py - 31 root - INFO - yaml file: params.yaml loaded successfully
[2025-02-27 01:30:03,829] common.py - 50 root - INFO - created directory at: artifacts
Found 116 images belonging to 2 classes.


c:\Users\User\anaconda3\envs\chick\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 40s 5s/step - accuracy: 0.7504 - loss: 1.5845
[2025-02-27 01:30:47,620] common.py - 64 root - INFO - json file saved at: scores.json


In [ ]:
# import tensorflow as tf
# from pathlib import Path
# import json

# class Evaluation:
#     def __init__(self, config):
#         """
#         Initializes the Evaluation class with the given config.
        
#         Args:
#             config (EvaluationConfig): Configuration object containing paths and parameters.
#         """
#         self.config = config
#         self.model = None  # Placeholder for model
#         self.valid_generator = None  # Placeholder for validation generator
#         self.score = None  # Placeholder for evaluation scores

#     def _valid_generator(self):
#         """
#         Creates the validation data generator using ImageDataGenerator.
#         """
#         datagenerator_kwargs = dict(rescale=1./255, validation_split=0.30)
#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],  # Resize images
#             batch_size=self.config.params_batch_size,  # Process images in batches
#             interpolation="bilinear"  # Use bilinear interpolation for resizing
#         )
        
#         # Initialize ImageDataGenerator
#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

#         # Create validation data generator
#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=str(self.config.training_data),  # Training data directory
#             subset="validation",  # Use the validation subset (30% of data)
#             shuffle=False,  # Keep the same order for validation
#             **dataflow_kwargs
#         )

#     @staticmethod
#     def load_model(path: Path) -> tf.keras.Model:
#         """
#         Loads a pre-trained model from a given path.
        
#         Args:
#             path (Path): Path to the saved model file.
        
#         Returns:
#             tf.keras.Model: Loaded Keras model.
#         """
#         return tf.keras.models.load_model(path)

#     def evaluation(self):
#         """
#         Loads the model and evaluates it on the validation dataset.
#         """
#         # Load the model correctly
#         self.model = Evaluation.load_model(self.config.path_of_model)

#         # Generate the validation data
#         self._valid_generator()

#         # Evaluate the model on validation data
#         self.score = self.model.evaluate(self.valid_generator)  # Fixed reference to model

#     def save_score(self):
#         """
#         Saves the evaluation scores (loss & accuracy) in a JSON file.
#         """
#         scores = {
#             'loss': self.score[0],  # First value is loss
#             'accuracy': self.score[1]  # Second value is accuracy
#         }

#         # Save the scores to a JSON file
#         with open("scores.json", "w") as f:
#             json.dump(scores, f, indent=4)  # Write scores as JSON
